In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Definir las agrupaciones
category_mapping = {
    0: [0, 1, 2, 3, 7, 9, 11, 22, 23, 26],  # Superficie transitable
    1: [29, 30, 31],  # Suelo blando
    2: [18, 50],  # Hierba baja
    3: [51, 52],  # Hierba alta
    4: [17, 16, 30],  # Vegetación densa
    5: [28, 27, 62],  # Árboles
    6: [4, 41, 42, 39, 43, 44, 25, 48, 45, 46, 47, 6, 19],  # Obstáculos urbanos y rurales
    7: [12, 15, 34, 20, 13, 32, 49, 35, 36, 37, 63, 57],  # Vehículos y elementos en movimiento
    8: [1, 55, 56, 58, 60, 61],  # Objetos pequeños y dispersos
    9: [53, 54, 2]  # Ambiente
}

category_mapping[6] += [5, 8, 10, 14, 21, 24, 33, 38, 40, 59]  # Añadir a Obstáculos urbanos y rurales